In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import requests

In [223]:
PATH = r"C:\Users\BILAL\Projects\LinkedInScraper\chromedriver.exe"
driver = webdriver.Chrome(PATH)

In [10]:
chrome_options = Options()
chrome_options.add_argument('--no-sandbox')

In [224]:
driver.get("https://www.linkedin.com/login")

In [225]:
driver.find_element_by_id("username").send_keys('donnybegins@gmail.com')
password = driver.find_element_by_id("password")
password.send_keys('myaccount')
password.send_keys(Keys.RETURN)

In [39]:
password

<selenium.webdriver.remote.webelement.WebElement (session="919725d6a590a3a96a278756515b2e17", element="6c70517a-2b2f-4afe-bef7-24a37bcc3dc6")>

In [35]:
driver.current_url

'https://www.linkedin.com/feed/'

In [206]:
driver.get("https://www.linkedin.com/feed/")

In [226]:
search = driver.find_element_by_class_name("search-global-typeahead__input ")

In [227]:
search.send_keys("MERN")

In [48]:
print(search)

<selenium.webdriver.remote.webelement.WebElement (session="919725d6a590a3a96a278756515b2e17", element="fa25496a-d4ab-4f84-bc70-c04d0fbbe4b5")>


In [228]:
search.send_keys(Keys.RETURN)

'https://www.linkedin.com/search/results/all/?keywords=MERN&origin=GLOBAL_SEARCH_HEADER&page=1'

In [181]:
driver.execute_script("window.history.go(-1)")


In [75]:
driver.page_source

'<html lang="en" class="theme theme--mercado artdeco windows"><head>\n    <script type="text/javascript" async="" charset="utf-8" id="utag_128" src="https://platform.linkedin.com/litms/vendor/google/gtag-adwords.js?id=AW-867846157"></script><script async="" src="https://platform.linkedin.com/js/analytics.js"></script><script type="application/javascript">!function(i,n){void 0!==i.addEventListener&&void 0!==i.hidden&&(n.liVisibilityChangeListener=function(){i.hidden&&(n.liHasWindowHidden=!0)},i.addEventListener("visibilitychange",n.liVisibilityChangeListener))}(document,window);</script>\n\n    <title>(9) "MERN" | Search | LinkedIn</title>\n\n    <meta charset="utf-8">\n    <meta http-equiv="X-UA-Compatible" content="IE=edge">\n    <meta name="asset-url" class="mercado-icons-sprite" id="artdeco-icons/static/images/sprite-asset" content="https://static-exp1.licdn.com/sc/h/21m48fftxrxil7c8lck4bcp41">\n    <meta name="description" content="">\n    <meta name="google" content="notranslate">

In [229]:
driver.find_element_by_xpath("//div[@id='a']//a[@class='click']"

SyntaxError: unexpected EOF while parsing (<ipython-input-229-badefe83004c>, line 1)

In [230]:
people = driver.find_element_by_xpath("//button[@aria-label='People']")
people.click()

In [231]:
container = driver.find_element_by_xpath("//button[@aria-label='Locations filter. Clicking this button displays all Locations filter options.']")
container

<selenium.webdriver.remote.webelement.WebElement (session="bb9580031267e6ceeb9cf42029ec66f7", element="4688d549-6786-43e8-9cb6-a92cf4d710ef")>

In [232]:
container.click()

In [233]:
country = driver.find_element_by_xpath("//label[@aria-label='Filter by India']")
country

<selenium.webdriver.remote.webelement.WebElement (session="bb9580031267e6ceeb9cf42029ec66f7", element="55229bbd-e61f-489c-843a-338869fb4a98")>

In [234]:
country.click()

In [220]:
pagination = driver.current_url
pagination

'https://www.linkedin.com/search/results/people/?geoUrn=%5B%22102713980%22%5D&keywords=MERN&origin=FACETED_SEARCH'

In [221]:
pagination = f"{pagination}&page=1"
pagination

'https://www.linkedin.com/search/results/people/?geoUrn=%5B%22102713980%22%5D&keywords=MERN&origin=FACETED_SEARCH&page=1'

In [235]:
li = driver.find_elements_by_xpath("//div[@class='artdeco-hoverable-content__shell']//button[@aria-label='Apply selected filter and show results']")
li[1].click()

In [248]:
result_list = driver.find_elements_by_xpath("//li[@class='reusable-search__result-container ']")
len(result_list)

10

In [257]:
for enu,result in enumerate(result_list):
    try:
        connect = result.find_element_by_xpath("//button[@class='artdeco-button artdeco-button--2 artdeco-button--secondary ember-view']")
        print(connect)
        connect.click()
    except Exception as e:
        print("Connection request already sent")
        print(e)
        
    if enu == 1:
        break

<selenium.webdriver.remote.webelement.WebElement (session="bb9580031267e6ceeb9cf42029ec66f7", element="a07c7ed4-cf2b-436f-9182-5f9d841b52d9")>
Connection request already sent
Message: element click intercepted: Element <button id="ember2410" class="artdeco-button artdeco-button--2 artdeco-button--secondary ember-view">...</button> is not clickable at point (759, 143). Other element would receive the click: <p class="t-14 t-black--light pb4">...</p>
  (Session info: chrome=87.0.4280.141)

<selenium.webdriver.remote.webelement.WebElement (session="bb9580031267e6ceeb9cf42029ec66f7", element="a07c7ed4-cf2b-436f-9182-5f9d841b52d9")>
Connection request already sent
Message: element click intercepted: Element <button id="ember2410" class="artdeco-button artdeco-button--2 artdeco-button--secondary ember-view">...</button> is not clickable at point (759, 143). Other element would receive the click: <p class="t-14 t-black--light pb4">...</p>
  (Session info: chrome=87.0.4280.141)



In [250]:
add_note = driver.find_element_by_xpath("//button[@aria-label='Add a note']")
add_note

<selenium.webdriver.remote.webelement.WebElement (session="bb9580031267e6ceeb9cf42029ec66f7", element="0463ea60-899f-4db3-ad35-1ca97fc95150")>

In [251]:
add_note.click()

In [252]:
add_note = driver.find_element_by_xpath("//textarea[@name='message']")
add_note

<selenium.webdriver.remote.webelement.WebElement (session="bb9580031267e6ceeb9cf42029ec66f7", element="057420b4-5d41-4dcc-850e-590b8b9ae5a0")>

In [253]:
note = 'I have a job opportunity for you'

In [254]:
add_note.send_keys(note)

In [255]:
send_req = driver.find_element_by_xpath("//button[@aria-label='Send now']")
send_req

<selenium.webdriver.remote.webelement.WebElement (session="bb9580031267e6ceeb9cf42029ec66f7", element="cbc62d2f-d7e1-46ee-963e-a051bc64a38d")>

In [256]:
send_req.click()